# RAG 구성 - 26.02.13 기준
### langchain version
- langchain                                1.2.10
- langchain-anthropic                      1.3.3
- langchain-chroma                         1.1.0
- langchain-classic                        1.0.1
- langchain-community                      0.4.1
- langchain-core                           1.2.11
- langchain-ollama                         1.0.1
- langchain-openai                         1.1.7
- langchain-openapi                        0.1.1
- langchain-text-splitters                 1.1.0
langchainhub                             0.1.21
### claude model + ollama embedding -> ollama model + ollama embedding
### openai, cluade 내부망 연결 실패 이슈
### 1. 문서 로드

In [ ]:
%pip install langchain-community docx2txt

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader

#1. 문서 로드
loader = Docx2txtLoader("tax.docx")
documents = loader.load() #테스트

len(documents)

### 2. 텍스트 분할

In [ ]:
%pip install langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

#1. 문서 로드
loader = Docx2txtLoader("tax.docx")

#2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

#로드 후 분할 작업 > 결과는 리스트 형태
document_list = loader.load_and_split(text_splitter=text_splitter)

print(len(document_list))

In [ ]:
document_list[0]

### 3. 임베딩 (ollama)

In [ ]:
from langchain_ollama.embeddings import OllamaEmbeddings

#before > terminal command: ollama pull nomic-embed-text
embeddings = OllamaEmbeddings(model="nomic-embed-text")

### 4. 벡터 DB (chroma)

In [ ]:
# chroma - in-memory vector db
%pip install langchain-chroma

In [ ]:
from langchain_chroma import Chroma

# 분할한 문서와 임베딩 모델을 이용해 벡터 DB 생성
database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    persist_directory="./chroma",
    collection_name="chroma_tax",
)

In [ ]:
query = "Calculate the income tax for a South Korean office worker earning 50 million won annually please."
docs = database.similarity_search(query, k=1) #유사도 검색 (Default: 4개 문서 가져옴)

### 5. 검색 (ollama) 

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
  model="llama3.2", #30억 파라미터 (기본) / llama3.2:1b은 10억 파라미터 (rag 작업은 안됨)
  num_ctx=2048, # 컨텍스트 제한
  num_predict=256, #출력 제한
  timeout=30,
  ) #llama2

In [ ]:
prompt = """
[Identity] You are a tax expert. 
Based on the following context, answer the question.

Context: {docs}
Question: {query}
"""

answer = llm.invoke(prompt.format(docs=docs, query=query))

### 답변 번역
소득세를 계산하려면 먼저 연간 소득을 기준으로 직원이 어느 범주에 속하는지 판단해야 합니다.
직원은 연간 5천만원(50,000,000원)을 벌고 있습니다.
제공된 문맥에 따르면, 이 금액은 "기타 소득" 범주에 해당하며, 세율은 20%의 100%, 즉 0.20입니다.
따라서 직원의 소득세는 다음과 같습니다:
소득세 = 연간 소득 x 소득세율
= 50,000,000 x 0.20
= 10,000,000원
그러므로 연봉 5천만원을 받는 한국 직장인의 소득세는 10,000,000원입니다.
그러나 만약 직원이 고정 연봉을 받고 있고, 그것이 고용주에 의해 월별 또는 분기별 같은 정기적으로 지급된다면, 그 금액은 각각 "월 소득" 또는 "분기 소득"에 해당할 것입니다.

### langchain hub import 이슈가 있어서 langsmith 로 변경 (apk key 필요)

langchainhub > langsmith

```
%pip install langsmith --break-system-packages
```

In [ ]:
import os
from langsmith import Client

LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY") #.env 파일에 key값 명시함
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("rlm/rag-prompt")

### langchain.chains 모듈 이슈가 있어서 langchain-classic 패키지 이동
langchain.chains > langchain_classic.chains

```
%pip install langchain-classic
```

In [ ]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={ "prompt": prompt }
)

In [ ]:
ai_message = qa_chain({"query" : query})

## 클로드 모델

In [ ]:
%pip install langchain-anthropic

In [ ]:
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv

load_dotenv() #load the .env file

llm = ChatAnthropic(model="claude-3-5-haiku-20241022") #빠르고 저렴 $0.8/$4 per 1M tokens

### 클로드에서는 임베딩을 제공하지 않아서 openai, ollma 임베딩으로 진행 필요
### 아래 임베딩+벡터DB 는 동일

In [ ]:
from langchain_ollama.embeddings import OllamaEmbeddings

#terminal command: ollama pull nomic-embed-text
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
)

In [ ]:
query = "Calculate the income tax for a South Korean office worker earning 50 million won annually please."
docs = database.similarity_search(query,k=2) #유사도 검색 (Default: 4개 문서 가져옴)

In [ ]:
docs

In [ ]:
prompt = """
[Identity] You are a tax expert. 
Based on the following context, answer the question.

Context: {docs}
Question: {query}
"""

answer = llm.invoke(prompt.format(docs=docs, query=query))

In [ ]:
%pip list | grep langchain